In [1]:
# __all__ = ['processExcel', 'plotRawData']
# class firstProcess:

import matplotlib.pyplot as plt
import pandas as pd
import numpy.matlib

class SinglePlate:
#     def __init__(self, plateName):
#         self.name = plateName
    def processExcel(df, *tecan): # if use spark, needs to specify
#         breakpoint()
        startidx = df.index[df[0].str.contains('Cycle Nr.', na=False)].tolist()[0]
        #print(tecan)
        if len(tecan)==0:
            endidx = df.index[df[0].str.contains('End Time:', na=False)].tolist()[0]
        elif tecan[0]=='spark':
            endidx = df.index[df[0].str.contains('End Time', na=False)].tolist()[1]
        else:
            print('Morese and Hildi is default. If use spark, enter "spark" \n')   
                        
        df1 = df.iloc[startidx:endidx,:]
        df1 = df1.dropna(how="all")
        df1.columns = df1.iloc[0,:]
        df1 = df1.drop(df1.index[0])
        df1 = df1.drop(columns=['Cycle Nr.','Temp. [°C]'])
#         print(df1.iloc[0,])
        df1.rename(columns={'Time [s]':'Time'},inplace=True)        
        df1.iloc[:,0] = df1.iloc[:,0] / 3600
        x=list(range(df1.shape[0]))
        df1.index = x
        return(df1)
    
    def processExcel_2(df, tecan): # if use spark, needs to specify
#         breakpoint()
        startidx = df.index[df[0].str.contains('Cycle Nr.', na=False)].tolist()[0]
        #print(tecan)
        if len(tecan)==0:
            endidx = df.index[df[0].str.contains('End Time:', na=False)].tolist()[0]
        elif tecan=='spark':
            endidx = df.index[df[0].str.contains('End Time', na=False)].tolist()[1]
        else:
            print('Morese and Hildi is default. If use spark, enter "spark" \n')   
                        
        df1 = df.iloc[startidx:endidx,:]
        df1 = df1.dropna(how="all")
        df1.columns = df1.iloc[0,:]
        df1 = df1.drop(df1.index[0])
        df1 = df1.drop(columns=['Cycle Nr.','Temp. [°C]'])
#         print(df1.iloc[0,])
        df1.rename(columns={'Time [s]':'Time'},inplace=True)        
        df1.iloc[:,0] = df1.iloc[:,0] / 3600
        x=list(range(df1.shape[0]))
        df1.index = x
        return(df1)
    
    def nameWells(df1, subNames):  # rename the column of growthcurve wells using substrate name
        a = list(df1.columns.values.tolist()) 
        #     print(a)
        for i in range(1, len(a)):
#             print(i)
#             print(a[i])
            # find the index of a[i] (well name) in the subName dataframe to match the substrate name
            B = subNames.Metabolite[subNames.index[subNames['wellname']==a[i]]].tolist()[0]
        #     print(i)
        #     print(a[i])
        #     print(subNames.iloc[subNames.index[subNames['wellname']==a[i]],:])
            df1.rename(columns={a[i]:B},inplace=True)
        return(df1)
#     def nameWellsByRowCol(df1, rows, Cols):

    def renameWellByRowCol(df, row, col, newName):
        # Example:
        # row1 = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H')
        # lacDF = SinglePlate.renameWellByRowCol(lacDF, row1,['1', '12'], 'blank')
        # eg. lacDF = SinglePlate.renameWellByRowCol(lacDF, row1,['11'], 'blank')
        a = list(df.columns.values.tolist())
        for r in row:
            for j in col:
                well = r+str(j)
                if well in a:
                    idx = a.index(well)
                    df.rename(columns={a[idx]:newName},inplace=True)
                else:
                    print(well + ' is not in dataframe title')
        return(df)
    
    def subtractFirstPoint(df):   # use the average of first two hours to subtract
        d = df.iloc[0:12]
        dm = d.mean(axis = 0)
        dm[0] = 0
        dr = np.matlib.repmat(dm, df.shape[0], 1)
        df = df -dr
        return(df)
    
    def subtractNegativeControl(df):   # if the plate contains negative control, subtract it from all conditions
        dfcolnames = df.columns.values.tolist()
        if 'Negative Control' in dfcolnames:
            x = dfcolnames.index('Negative Control')
            dm = df.iloc[:, x]
            dr = np.matlib.repmat(dm, df.shape[1],1)
            dr = np.transpose(dr)
            dr[:,0] = 0
            df = df - dr
        else:
            print('Negative control not found')
        return(df)     
    
    def keepColForEachStrain(df1, strain, col2keep):
        a = list(df1.columns.values.tolist()) 
        B=[]
        for i in range(len(a)):   
            if a[i] == strain:
                B.append(i)
#         print(B)
        B=list(np.array(B)[col2keep])
#         print(B)
        straindf=df1.iloc[:, B];
        return(straindf)
    
    
    def creatCleanTable(df, platename, strain, condition, tecan, rep):
        # G2=SinglePlate.creatCleanTable(newG1, gcExcels[0], 'clinical', 'glycerol', 'Hildi', 'rep1')
        colnames = list(df.columns)
        ## create dataframe for mixed linear model
        df2=pd.melt(df, id_vars=['Time'], value_vars=colnames[1:len(colnames)], 
                var_name='metabolites', value_name='absorbance')
        df2['plate'] = platename
        df2['strain'] = strain
        df2['condition'] = condition
        df2['rep'] = rep
        df2['tecan'] = tecan
        return(df2)

    def plotRawData(df, platename):
        # df1.info()
#         breakpoint()
        df.apply(pd.to_numeric)
        plt.figure(figsize=(18,25))
        Max = df.iloc[-1, 1:97].max()
#         print(Max)
#         for i in range(1, 97):
#             plt.subplot(12, 8, i)
#             plt.scatter(df.iloc[:,0], df.iloc[:,i], s = 5)
#         #     plt.suptitle(df1.columns[i])
#             plt.ylim(0, Max*1.2)
#             plt.gca().set_title(df.columns[i])
        for i in range(1,9):
            for j in range(1,13):
                plt.subplot(8, 12, (i-1)*12+j)
                plt.scatter(df.iloc[:,0], df.iloc[:,(j-1)*8+i], s = 5)
            #     plt.suptitle(df1.columns[i])
                plt.ylim(0, Max+0.4)
                plt.gca().set_title(df.columns[(j-1)*8+i])
        plt.suptitle(platename)
#         plt.figlegend( labels, loc = 'lower center', ncol=5, labelspacing=0. )
        plt.show()
#         plt.savefig(plate+'_log.pdf', transparent=True)
    
    def plotRawData_2(df, platename, colNumber):
        # df1.info()
#         breakpoint()
        df.apply(pd.to_numeric)
        plt.figure(figsize=(18,25))
        Max = df.iloc[-1, 1:97].max()
#         print(Max)
#         for i in range(1, 97):
#             plt.subplot(12, 8, i)
#             plt.scatter(df.iloc[:,0], df.iloc[:,i], s = 5)
#         #     plt.suptitle(df1.columns[i])
#             plt.ylim(0, Max*1.2)
#             plt.gca().set_title(df.columns[i])
        for i in range(1,9):
            for j in range(1,colNumber + 1):
                plt.subplot(8, 12, (i-1)*12+j)
                plt.scatter(df.iloc[:,0], df.iloc[:,(j-1)*8+i], s = 5)
            #     plt.suptitle(df1.columns[i])
                plt.ylim(0, Max+0.4)
                plt.gca().set_title(df.columns[(j-1)*8+i])
        plt.suptitle(platename)
#         plt.figlegend( labels, loc = 'lower center', ncol=5, labelspacing=0. )
        plt.show()
        
    def plotReplicateslog(dflist, platename, repnames):
#         breakpoint()
        plt.figure(figsize=(18,15))
        a = len(dflist)
        Max = np.zeros(a)
        for i in range(1,a+1):
            Max[i-1] = dflist[i-1].iloc[-1, 1:97].max()
        # m = max(Max1, Max2)*1.2
        m = np.amax(Max)*1.2
        jet = cm.get_cmap('jet', a)
        mycmp = jet(np.linspace(0, 1, a))
        # mycmp = cm.get_cmap('jet', a)
        for q in range(1, a+1):
            for i in range(1, 9):
                for j in range(1,13):
                    plt.subplot(8, 12, (i-1)*12+j)
                    plt.scatter(dflist[q-1].iloc[:,0], np.log10(dflist[q-1].iloc[:,(j-1)*8+i].astype(float)), color =mycmp[a-1], s = 3)
            #         plt.scatter(dflist[j-1].iloc[:,0], np.log10(dflist[j-1].iloc[:,i].astype(float)), color = 'orange', s = 2)
                plt.ylim(-1.1, np.log10(m*1.5))
                plt.gca().set_title(dflist[0].columns[(j-1)*8+i],size=10, color='blue', y=0.92)
                for ax in plt.gcf().axes:
                    try:
                        ax.label_outer()
                    except:
                        pass
        title = platename +repnames[0]+'_log: Reps';
        labels = repnames
        #         plt.legend(labels)
        plt.suptitle(title,size=20)
        plt.figlegend( labels, loc = 'upper center', ncol=3, labelspacing=0.5 )
        plt.savefig(platename+repnames[0]+'_log.pdf', transparent=True)
        plt.show()

In [16]:
class CompareStrains:
    def plot2strains(df1, df2, plate, name1, name2):
        plt.figure(figsize=(18,15))
        Max1 = df1.iloc[-1, 1:97].max()
        Max2 = df2.iloc[-1, 1:97].max()
        m = max(Max1, Max2)
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            plt.scatter(df1.iloc[:,0], df1.iloc[:,i], s = 5)
            plt.scatter(df2.iloc[:,0], df2.iloc[:,i], color = 'orange', s = 2)
            plt.ylim(0, m*1.2)
            plt.gca().set_title(df1.columns[i])
        title = plate+': ' +name1+' vs '+ name2;
        labels = [name1, name2]
        plt.legend(labels)
        plt.suptitle(title)
        plt.figlegend( labels, loc = 'lower center', ncol=5, labelspacing=0. )
        plt.show()
        
    def plot2strainsRep(dflist1, dflist2, plate, name1, name2):
        a1 = len(dflist1)
        a2 = len(dflist2)
        Max1 = np.zeros((a1,1))
        Max2 = np.zeros((a2,1))
        Min1 = np.zeros((a1,1))
        Min2 = np.zeros((a2,1))
        for i in range(1,a1+1):
            Max1[i-1] = dflist1[i-1].iloc[-1, 1:97].max()
            Min1[i-1] = dflist1[i-1].iloc[-1, 1:97].min()
        for i in range(1,a2+1):
            Max2[i-1] = dflist2[i-1].iloc[-1, 1:97].max()
            Min2[i-1] = dflist2[i-1].iloc[-1, 1:97].min()   
        print(Max1)
        print(Max2)
        m = max(max(Max1), max(Max2))
        mn = min(min(Min1), min(Min2))    
        print(m)
        plt.figure(figsize=(18,15))
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            for j in range(1,max(a1, a2)+1):
                if a1 < j:
                    continue
                else:
                    plt.scatter(dflist1[j-1].iloc[:,0], dflist1[j-1].iloc[:,i], color ='orange', s = 3)
                    
                if a2 < j:
                    continue 
                else:                       
                    plt.scatter(dflist2[j-1].iloc[:,0], dflist2[j-1].iloc[:,i], color = 'green', s = 2)  
            plt.ylim(mn-0.05, m+0.1)
            plt.gca().set_title(dflist1[0].columns[i],size=10, color='blue', y=0.92)
            for ax in plt.gcf().axes:
                try:
                    ax.label_outer()
                except:
                    pass           
            
        title = plate+': ' +name1+' vs '+ name2;
        labels = [name1, name2]
#         plt.legend(labels)
        plt.suptitle(title)
        plt.figlegend( labels, loc = 'upper center', ncol=5, labelspacing=0. )
        plt.show()       
        
        
        
    def plot2strainslog(df1, df2, plate, name1, name2):
        plt.figure(figsize=(18,15))
        Max1 = df1.iloc[-1, 1:97].max()
        Max2 = df2.iloc[-1, 1:97].max()
        m = max(Max1, Max2)*1.1
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            plt.scatter(df1.iloc[:,0], np.log10(df1.iloc[:,i].astype(float)), s = 3)
            plt.scatter(df2.iloc[:,0], np.log10(df2.iloc[:,i].astype(float)), color = 'red', s = 2)
            plt.ylim(-1.2, np.log10(m)+0.05)
            plt.gca().set_title(df1.columns[i],size=10, color='blue', y=0.92)
            for ax in plt.gcf().axes:
                try:
                    ax.label_outer()
                except:
                    pass
        title = plate +': ' +name1+' vs '+ name2;
        labels = [name1, name2]
#         plt.legend(labels)
        plt.suptitle(title,size=20)
        plt.figlegend( labels, loc = 'upper center', ncol=3, labelspacing=0.5 )
        plt.savefig(plate+'_log.pdf', transparent=True)
        plt.show()
        
    def plot2strainsReplog(dflist1, dflist2, plate, name1, name2): #the two strains have the same # of reps
        plt.figure(figsize=(18,15))
        a = len(dflist1)
        b = 2
        Max = np.zeros((a,2))
        Min = np.zeros((a,2))
        for i in range(1,a+1):
            Max[i-1, 0] = dflist1[i-1].iloc[-1, 1:97].max()
            Max[i-1, 1] = dflist2[i-1].iloc[-1, 1:97].max()
            Min[i-1, 0] = np.log10(dflist1[i-1].iloc[-1, 1:97].min())
            Min[i-1, 1] = np.log10(dflist2[i-1].iloc[-1, 1:97].min())
        # m = max(Max1, Max2)*1.2
        m = np.amax(Max)
        mn = np.amin(Min)
        jet = cm.get_cmap('jet', b)
        mycmp = jet(np.linspace(0, 1, b))
        # mycmp = cm.get_cmap('jet', a)
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            for j in range(1,a+1):    
                plt.scatter(dflist1[j-1].iloc[:,0], np.log10(dflist1[j-1].iloc[:,i].astype(float)), color ='orange', s = 3)
                plt.scatter(dflist2[j-1].iloc[:,0], np.log10(dflist2[j-1].iloc[:,i].astype(float)), color = 'green', s = 2)
            plt.ylim(-1.2, np.log10(m)+0.1)
            plt.gca().set_title(dflist1[0].columns[i],size=10, color='blue', y=0.92)
            for ax in plt.gcf().axes:
                try:
                    ax.label_outer()
                except:
                    pass
        title = plate+': ' +name1+' vs '+ name2;
        labels = [name1, name2]
        #         plt.legend(labels)
        plt.suptitle(title,size=20)
        plt.figlegend( labels, loc = 'upper center', ncol=3, labelspacing=0.5 )
        plt.savefig(plate+'_log.pdf', transparent=True)
        plt.show()
        
    def plot2strainsAllRepConlog(dflist1, dflist2, dflist1con, dflist2con, plate, name1, name2, condition): #the two strains have the same # of reps
        plt.figure(figsize=(18,15))
        a = len(dflist1)
        b = 4
        Max = np.zeros((a,b))
        Min = np.zeros((a,2))
        for i in range(1,a+1):
            if len(dflist1)< i:
                Max[i-1, 0] = 0;
                Max[i-1, 1] = 0;
                Min[i-1, 0] = 0;
                Min[i-1, 1] = 0;
            else:
                Max[i-1, 0] = dflist1[i-1].iloc[-1, 1:97].max()
                Max[i-1, 1] = dflist2[i-1].iloc[-1, 1:97].max()
                Min[i-1, 0] = np.log10(dflist1[i-1].iloc[-1, 1:97].min())
                Min[i-1, 1] = np.log10(dflist2[i-1].iloc[-1, 1:97].min())
            if len(dflist1con) < i:
                Max[i-1, 2] = 0;
                Max[i-1, 3] = 0;
                Min[i-1, 0] = 0;
                Min[i-1, 1] = 0;
            else:
                Max[i-1, 2] = dflist1con[i-1].iloc[-1, 1:97].max()
                Max[i-1, 3] = dflist2con[i-1].iloc[-1, 1:97].max()
                Min[i-1, 0] = np.log10(dflist1con[i-1].iloc[-1, 1:97].min())
                Min[i-1, 1] = np.log10(dflist2con[i-1].iloc[-1, 1:97].min())
        # m = max(Max1, Max2)*1.2
        m = np.amax(Max)
        mn = np.amin(Min)
        jet = cm.get_cmap('jet', b)
        mycmp = jet(np.linspace(0, 1, b))
        # mycmp = cm.get_cmap('jet', a)
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            for j in range(1,a+1):
                if len(dflist1) < j:
                    continue
                else:
                    plt.scatter(dflist1[j-1].iloc[:,0], np.log10(dflist1[j-1].iloc[:,i].astype(float)), color ='orange', s = 3)
                    plt.scatter(dflist2[j-1].iloc[:,0], np.log10(dflist2[j-1].iloc[:,i].astype(float)), color = 'green', s = 2)
                    
                if len(dflist1con) < j:
                    continue 
                else:
                    plt.scatter(dflist1con[j-1].iloc[:,0], np.log10(dflist1con[j-1].iloc[:,i].astype(float)), color ='red', s = 3)
                    plt.scatter(dflist2con[j-1].iloc[:,0], np.log10(dflist2con[j-1].iloc[:,i].astype(float)), color = 'cyan', s = 2)
            plt.ylim(np.log10(0.1)-0.2, np.log10(m)+0.05)
            plt.gca().set_title(dflist1[0].columns[i],size=10, color='blue', y=0.92)
            for ax in plt.gcf().axes:
                try:
                    ax.label_outer()
                except:
                    pass
        title = plate+': ' +name1+' vs '+ name2 + ' with or without '+condition;
        labels = [name1, name2, name1+condition, name2+condition]
        #         plt.legend(labels)
        plt.suptitle(title,size=20)
        plt.figlegend( labels, loc = 'upper center', ncol=2, labelspacing=0.5 )
        plt.savefig(plate+condition+'all_log.pdf', transparent=True)
        plt.show()  
        
        
    def plot2strainsAllRepCon(dflist1, dflist2, dflist1con, dflist2con, plate, name1, name2, condition): #the two strains have the same # of reps
        plt.figure(figsize=(18,15))
        a = len(dflist1)
        b = 4
        Max = np.zeros((a,b))
        Min = np.zeros((a,2))
        for i in range(1,a+1):
            if len(dflist1)< i:
                Max[i-1, 0] = 0;
                Max[i-1, 1] = 0;
                Min[i-1, 0] = 0;
                Min[i-1, 1] = 0;
            else:
                Max[i-1, 0] = dflist1[i-1].iloc[-1, 1:97].max()
                Max[i-1, 1] = dflist2[i-1].iloc[-1, 1:97].max()
                Min[i-1, 0] = dflist1[i-1].iloc[-1, 1:97].min()
                Min[i-1, 1] = dflist2[i-1].iloc[-1, 1:97].min()
            if len(dflist1con) < i:
                Max[i-1, 2] = 0;
                Max[i-1, 3] = 0;
                Min[i-1, 0] = 0;
                Min[i-1, 1] = 0;
            else:
                Max[i-1, 2] = dflist1con[i-1].iloc[-1, 1:97].max()
                Max[i-1, 3] = dflist2con[i-1].iloc[-1, 1:97].max()
                Min[i-1, 0] = dflist1con[i-1].iloc[-1, 1:97].min()
                Min[i-1, 1] = dflist2con[i-1].iloc[-1, 1:97].min()
        # m = max(Max1, Max2)*1.2
        m = np.amax(Max)
        mn = np.amin(Min)
        jet = cm.get_cmap('jet', b)
        mycmp = jet(np.linspace(0, 1, b))
        # mycmp = cm.get_cmap('jet', a)
        for i in range(1, 97):
            plt.subplot(12, 8, i)
            for j in range(1,a+1):
                if len(dflist1) < j:
                    continue
                else:
                    plt.scatter(dflist1[j-1].iloc[:,0], dflist1[j-1].iloc[:,i], color ='orange', s = 3)
                    plt.scatter(dflist2[j-1].iloc[:,0], dflist2[j-1].iloc[:,i], color = 'green', s = 2)

                if len(dflist1con) < j:
                    continue 
                else:
                    plt.scatter(dflist1con[j-1].iloc[:,0], dflist1con[j-1].iloc[:,i], color ='red', s = 3)
                    plt.scatter(dflist2con[j-1].iloc[:,0], dflist2con[j-1].iloc[:,i], color = 'cyan', s = 2)
            plt.ylim(mn-0.05, m+0.05)
            plt.gca().set_title(dflist1[0].columns[i],size=10, color='blue', y=0.92)
            for ax in plt.gcf().axes:
                try:
                    ax.label_outer()
                except:
                    pass
        title = plate+': ' +name1+' vs '+ name2 + ' with or without '+condition;
        labels = [name1, name2, name1+condition, name2+condition]
        #         plt.legend(labels)
        plt.suptitle(title,size=20)
        plt.figlegend( labels, loc = 'upper center', ncol=2, labelspacing=0.5 )
        plt.savefig(plate+condition+'OD600.pdf', transparent=True)
        plt.show()  